In [1]:
from classes import *
from helper_functions import *

import pandas as pd
import numpy as np
import datetime, time
import random
random.seed(2022)

# for configurations
from dotenv import load_dotenv
import sys, os, re, math
load_dotenv()

# for distance matrix
import gmaps
import googlemaps
from haversine import haversine
from scipy.spatial.distance import cdist

# for routing
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
## DATA HANDLING
file_path = 'data/processed_data_10.csv'
plan_date = datetime.date.today() # or you can do datetime.date(year,month,day)
num_vehicles = 4
ins_ends_coords = [(-34.8218243,138.7292797),(-34.8104796,138.6111791),(-34.8938435,138.6918266),(-34.7810071,138.6461490)] # Set the last installer which is a team .. its endlocation to factory
data = pd.read_csv(file_path) # Reading the csv

In [3]:
def solve(data,plan_date,num_vehicles,ins_ends_coords):
    
    ## CONFIGURATION 
    data = data.copy(deep=True)
    API_KEY = os.getenv("API_KEY") # replace api key with your own here
    FACTORY_GEO_COORD = os.getenv("FACTORY_GEO_COORD") 
    factory_coord = list(map(float,FACTORY_GEO_COORD.split(',')))
    gmaps.configure(api_key=API_KEY)
    
    ## FILTERING ONLY 1 INSTALLER JOBS WITH STATUS = None
    jobs_installer_1 = data[(data["installers_required"].notnull()) & pd.isnull(data["status"])].reset_index(drop=True)
    
    ## PROCESSING DATA FOR VRP USE
    df_pending = jobs_installer_1[['id','Latitude','Longitude']]
    new_row = pd.DataFrame({'id':0, 'Latitude':factory_coord[0],'Longitude':factory_coord[1]}, index =[0]) # Inserting factory location to top
    df_pending = pd.concat([new_row, df_pending]).reset_index(drop=True)
    df_pending.set_index('id', inplace=True)
    df_pending['Latitude'] = df_pending['Latitude'].astype(float)
    df_pending['Longitude'] = df_pending['Longitude'].astype(float)

    for i in range(len(ins_ends_coords)):  # Inserting installers' end locations to df_pending
        ins_id = i+1
        df_pending.loc[ins_id] = [ins_ends_coords[i][0],ins_ends_coords[i][1]]
        
    # SETTING UP variables and figures for VISUALISATION
    jobs, ins_ends = [],[]
    for i, row in df_pending.iterrows():
        if i == 0:
            continue
        elif i//100 == 0: # Expecting max 100 installers for the program and job_ids no less than 100
            ins_end = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
            ins_ends.append(ins_end)
        else:
            job = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
            jobs.append(job)

    factory = {'location': (factory_coord[0],factory_coord[1])}
    factory_layer = gmaps.symbol_layer([factory['location']], hover_text='Factory', info_box_content='Factory', fill_color='white', stroke_color='red', scale=6)

    job_locations = [job['location'] for job in jobs]
    job_labels = [job['id'] for job in jobs]
    jobs_layer = gmaps.symbol_layer(
        job_locations, hover_text=job_labels, fill_color='white', stroke_color='black', scale=3
    )

    ins_end_locations = [ins_end['location'] for ins_end in ins_ends]
    ins_labels = [ins_end['id'] for ins_end in ins_ends]
    ins_ends_layer = gmaps.symbol_layer(
        ins_end_locations, hover_text=ins_labels, fill_color='white', stroke_color='red', scale=3
    )

    fig = gmaps.figure()
    fig.add_layer(factory_layer)
    fig.add_layer(jobs_layer)
    fig.add_layer(ins_ends_layer)
    # fig
    
    ## DEFINING PENALTIES AND JOB_DURATIONS(DEMANDS)
    demands, penalties = [0],[0]
    pref_dates, pref_days, pref_installers=[None],[None],[None] # first values for depot 
    pref_time_windows=[None]
    installers_req=[None]
    ## DEMANDS 
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'expected_job_time']):
            demands.append(int(60))
            continue
        demands.append(int(jobs_installer_1.loc[i,'expected_job_time']))

    ## DUE_DATE_DIFFERENCE    
    for i in range(len(jobs)):
        due_date = datetime.datetime.strptime(jobs_installer_1.loc[i,'est_installation_date'], '%d/%m/%Y').date()
        curr_date = datetime.date.today()
        due_date_diff = (due_date-curr_date).days
        penalties.append(due_date_diff)

    ## CUSTOMER PREFERRED DATE
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'pref_date']):
            pref_dates.append(None)
            continue
        pref_date_to_append = datetime.datetime.strptime(jobs_installer_1.loc[i,'pref_date'], '%d/%m/%Y').date()
        pref_dates.append(pref_date_to_append)

    ## CUSTOMER PREFERRED DAY
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'pref_day']):
            pref_days.append(None)
            continue
        pref_day_to_append = int(jobs_installer_1.loc[i,'pref_day'])
        pref_days.append(pref_day_to_append)

    ## CUSTOMER PREFERRED INSTALLER
    for i in range(len(jobs)):
        if np.isnan(jobs_installer_1.loc[i,'pref_installer']):
            pref_installers.append(None)
            continue
        pref_installers.append(int(jobs_installer_1.loc[i,'pref_installer']))

    ## CUSTOMER PREFERRED TIME WINDOWS
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'pref_time_window']):
            pref_time_windows.append(None)
            continue
        curr_pref_time_window = jobs_installer_1.loc[i,'pref_time_window'].split(',')
        curr_pref_time_window[0] = time_to_minutes(curr_pref_time_window[0])
        curr_pref_time_window[1] = time_to_minutes(curr_pref_time_window[1])
        pref_time_windows.append(curr_pref_time_window)
    
    ## MULTIPLE INSTALLERS
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'installers_required']):
            installers_req.append(None)
            continue
        installers_req.append(int(jobs_installer_1.loc[i,'installers_required']))

    ## END LOCATIONS OF INSTALLERS
    end_locations = []
    for i in range(num_vehicles):
        end_locations.append(len(jobs)+i+1)

    dist_matrix,time_matrix = get_distance_time_matrices(df_pending)
    job_ids = df_pending.index.tolist()
    
    routes, total_distance, total_load, job_times = solve_vrp_for(time_matrix, num_vehicles, demands, penalties, end_locations, pref_dates, pref_days, pref_installers, pref_time_windows, plan_date, job_ids, installers_req)
    
    ## UNCOMMENT THE CODE BELOW TO VISUALIZE THE ROUTES
    if routes:
        map_solution(factory, jobs, ins_ends, routes, fig)
    else:
        print('No solution found.') 
    display(fig)
    
    if job_times==None or (not job_times):
        pass
    else:
        for job_id, out_values in job_times.items():
            start_time_str = "{:02d}:{:02d}".format((out_values['start_time']//60)+8, out_values['start_time']%60)
            end_time_str = "{:02d}:{:02d}".format((out_values['end_time']//60)+8, out_values['end_time']%60)
            data.loc[data['id'] == job_id, ['installation_date','arrival_start_time','arrival_end_time','installer_ids','status']] = plan_date.strftime("%d/%m/%Y"), start_time_str, end_time_str, str(out_values['installer_id']), 'Scheduled'
    
    return routes, total_distance, total_load, job_times, data

In [4]:
day1 = next_working_date(plan_date)
routes1, total_distance1, total_load1, job_times1, data_output_day1 = solve(data,day1,num_vehicles,ins_ends_coords)

1 0
2 14300
3 0
4 0
5 0
6 0
7 0
8 14500
9 0
10 11400
11 0
12 0
13 0
14 14100
Planning Date: 2023-05-26
Dropped nodes: 1,3,4,5,6,7,9,11,12,13,
Route for vehicle 0:
 ID:0, Time:(0-0) ->  Ins_End 0 @ (0min)
Total time spent: 0min
Total job time spent: 0min

Route for vehicle 1:
 ID:0, Time:(0-0) ->  Ins_End 1 @ (0min)
Total time spent: 0min
Total job time spent: 0min

Route for vehicle 2:
 ID:0, Time:(0-0) ->  ID:2363, Time:(42-162) ->  Ins_End 2 @ (196min)
Total time spent: 196min
Total job time spent: 120min

Route for vehicle 3:
 ID:0, Time:(0-0) ->  ID:2745, Time:(17-137) ->  ID:2662, Time:(162-282) ->  ID:2608, Time:(293-413) ->  Ins_End 3 @ (459min)
Total time spent: 459min
Total job time spent: 360min

Total time spent of all routes: 655min
Total job_time spent of all routes: 480min
Empty route: 0
Empty route: 1


Figure(layout=FigureLayout(height='420px'))

In [5]:
day2 = next_working_date(day1)
routes2, total_distance2, total_load2, job_times2, data_output_day2 = solve(data_output_day1,day2,num_vehicles,ins_ends_coords)

1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 999999
9 0
10 999999
Planning Date: 2023-05-29
Dropped nodes: 1,2,3,4,5,6,7,9,
Route for vehicle 0:
 ID:0, Time:(0-0) ->  Ins_End 0 @ (0min)
Total time spent: 0min
Total job time spent: 0min

Route for vehicle 1:
 ID:0, Time:(0-0) ->  ID:2702, Time:(7-187) ->  Ins_End 1 @ (201min)
Total time spent: 201min
Total job time spent: 180min

Route for vehicle 2:
 ID:0, Time:(0-0) ->  ID:2719, Time:(32-212) ->  Ins_End 2 @ (220min)
Total time spent: 220min
Total job time spent: 180min

Route for vehicle 3:
 ID:0, Time:(0-0) ->  Ins_End 3 @ (0min)
Total time spent: 0min
Total job time spent: 0min

Total time spent of all routes: 421min
Total job_time spent of all routes: 360min
Empty route: 0
Empty route: 3


Figure(layout=FigureLayout(height='420px'))

In [6]:
day3 = next_working_date(day2)
routes3, total_distance3, total_load3, job_times3, data_output_day3 = solve(data_output_day2,day3,num_vehicles,ins_ends_coords)

1 0
2 0
3 0
4 0
5 0
6 0
7 999999
8 0
9 999999
Planning Date: 2023-02-13
Dropped nodes: 1,2,3,4,5,6,8,
Route for vehicle 0:
 ID:0, Time:(0-0) ->  Ins_End 0 @ (0min)
Total time spent: 0min
Total job time spent: 0min

Route for vehicle 1:
 ID:0, Time:(0-0) ->  ID:2702, Time:(7-187) ->  Ins_End 1 @ (201min)
Total time spent: 201min
Total job time spent: 180min

Route for vehicle 2:
 ID:0, Time:(0-0) ->  ID:2719, Time:(32-212) ->  Ins_End 2 @ (220min)
Total time spent: 220min
Total job time spent: 180min

Route for vehicle 3:
 ID:0, Time:(0-0) ->  Ins_End 3 @ (0min)
Total time spent: 0min
Total job time spent: 0min

Total time spent of all routes: 421min
Total job_time spent of all routes: 360min
Empty route: 0
Empty route: 3


Figure(layout=FigureLayout(height='420px'))

In [7]:
# ## GOOGLE DISTANCE MATRIX API
# cumul_dist,cumul_time=0,0
# for route in routes:
#     total_distance,total_time = 0,0
#     prev_node = None
#     curr_node = None
#     for node in routes[route]:
#         if prev_node==None:
#             prev_node = node
#             continue
#         curr_node = node
#         prev_coord = (df_pending.iloc[prev_node]['Latitude'],df_pending.iloc[prev_node]['Longitude'])
#         curr_coord = (df_pending.iloc[curr_node]['Latitude'],df_pending.iloc[curr_node]['Longitude'])
#         total_distance += get_distance(prev_coord,curr_coord)
#         total_time += get_travel_time(prev_coord,curr_coord)
# #         print(prev_coord, end='--')
#         prev_node = curr_node
# #     print(curr_coord)
#     print(total_distance/1000,total_time)
#     cumul_dist+=total_distance
#     cumul_time+=total_time
# print(cumul_dist/1000,cumul_time)

In [8]:
data_output_day3.to_csv('output/processed_data_3_out.csv',index=False)